In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from njl_corf import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]

In [ ]:
import datetime
import pandas as pd
from typing import Sequence

In [ ]:
%autoreload 2
from njl_corf import pyoscar

In [ ]:
oscar_data = pyoscar.read()

In [ ]:
statuses = sorted(list(set([b.data.status for b in oscar_data])))
print(statuses)

In [ ]:
def table_type_a(
    entries: Sequence[pyoscar.OscarEntry],
) -> pd.DataFrame:
    """
    Convert a sequence of OscarEntry objects into a Pandas DataFrame.
    """
    data = {
        "Satellite": [entry.satellite for entry in entries],
        "Space Agency": [entry.space_agency for entry in entries],
        "EOL": [entry.eol for entry in entries],
        "Service": [entry.service for entry in entries],
        "Bandwidth": [entry.bandwidth for entry in entries],
        "Polarization": [entry.polarization for entry in entries],
        "Launch": [entry.launch for entry in entries],
        "Service (aka Sensor)": [entry.service for entry in entries],
    }
    return pd.DataFrame(data)


def table_type_b(
    entries: Sequence[pyoscar.OscarEntry],
) -> pd.DataFrame:
    """
    Convert a sequence of OscarEntry objects into a Pandas DataFrame.
    """
    data = {
        "Satellite": [entry.satellite for entry in entries],
        "Sensor": [entry.service for entry in entries],
        "Launch": [entry.launch for entry in entries],
        "EOL": [entry.eol for entry in entries],
        "Center Frequency": [entry.nominal_frequency for entry in entries],
        "Bandwidth": [entry.bandwidth for entry in entries],
    }
    return pd.DataFrame(data)

In [ ]:
groups = {
    "Previous": ["Inacative", "Lost at Launch", "Presumably inactive", "Unclear"],
    "Current": ["Operational"],
    "Planned": ["Planned", "Commissioning", "Considered"],
}

In [ ]:
agenda_items = {
    "AI-1.1": (slice(50.2 * ureg.GHz, 50.4 * ureg.GHz), table_type_a),
    "AI-1.3a": (51.76 * ureg.GHz, table_type_a),
    "AI-1.3b": (slice(52.61 * ureg.GHz, 54.25 * ureg.GHz), table_type_a),
    "AI-2.3": (slice(10.6 * ureg.GHz, 10.7 * ureg.GHz), table_type_a),
    "AI-2.4": (slice(6425 * ureg.MHz, 7075 * ureg.MHz), table_type_b),
}

In [ ]:
for agenda_item, (frequency_range, table_function) in agenda_items.items():
    entries = oscar_data[frequency_range]
    tables = {
        key: table_function(
            [b.data for b in entries if b.data.status in item]
        ).sort_values(by="Satellite")
        for key, item in groups.items()
    }
    for key, table in tables.items():
        table.to_csv(
            f"wrc27-tables/{agenda_item.replace('.', '-')}_{key}.csv",
            encoding="utf-16",
            index=False,
            sep="\t",
        )